# RGI-07: Region 17 (Southern Andes)
##### F. Roura November 2021

Goal: compare L2 GLIMS files to original inventory to check possible errors in GLIMS ingestion.

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import subprocess
import os
import numpy as np
from utils import mkdir

## Files and storage paths

In [ ]:
# Region of interest
reg = 17

# go down from rgi7_scripts/workflow
data_dir = '../../rgi7_data/'

# Level 2 GLIMS files
l2_dir = os.path.join(data_dir, 'l2_sel_reg_tars')

# Output directories
output_dir = mkdir(os.path.join(data_dir, 'l3_rgi7a'))
output_dir_tar = mkdir(os.path.join(data_dir, 'l3_rgi7a_tar'))

# Original inventory for GLIMS check 
ref_reg_file = os.path.join(data_dir, 'l0_support_data', 'Shape_Inventario_de_Glaciares.zip') 

### Load the input data

In [ ]:
# Read L2 files from GLIMS
shp = gpd.read_file('tar://' + l2_dir + f'/RGI{reg:02d}.tar.gz/RGI{reg:02d}/RGI{reg:02d}.shp')

## List of submissions in GLIMS L2

In [ ]:
toprint = []
for subid in shp.subm_id.unique():
    s_loc = shp.loc[shp.subm_id == subid]
    s = ''
    for c in ['subm_id', 'analysts', 'src_date']:
        toprint = s_loc[c].unique()
        if c != 'src_date':
            s += ' ' + (str(toprint[0]))
        else:
            for d in toprint:
                s += ' ' + d[:4]
    print(s)

## Apply selection criteria to compare Glims data set to the original one

### Step 1: extract ingested inventory from GLIMS data and do a check

In [ ]:
#...extract l2 from GLIMS based on 'sumb_id'
RGI_ss = shp.loc[shp['subm_id'] == 730].copy() #barcaza

In [ ]:
# Write out selection in intermediate shape files for manual GIS review
#tmp_output_dir = mkdir(os.path.join(data_dir, 'l0_tmp_data', 'rgi17_inventories'))
#tmp_output_dir_tar = mkdir(os.path.join(data_dir, 'l0_tmp_data'))
#RGI_ss.to_file(tmp_output_dir + f'/subm_{int(730):03d}.shp')
#print('Taring...')
#print(subprocess.run(['tar', '-zcvf', f'{tmp_output_dir_tar}/rgi17_inventories.tar.gz', '-C', 
#                      os.path.join(data_dir, 'l0_tmp_data'), 'rgi17_inventories']))

#### load reference data (here original inventory)

In [ ]:
# Just to know the name of the file to open from zip
import zipfile
with zipfile.ZipFile(ref_reg_file, "r") as z:
    for f in z.filelist:
        if 'Inventario_de_Glaciares.shp' in f.filename:
            if '.shp.xml' in f.filename:
                break
            else:
                fname = f.filename

# load reference data
ref_odf = gpd.read_file('zip://' + ref_reg_file + '/' + fname)

#### Number of elements (differences do not necessarily depict major problems)

In [ ]:
print('Number of glaciers in new RGI subset:', len(RGI_ss))
print('Number of glaciers in reference data:', len(ref_odf))
print('Difference:', len(RGI_ss)-len(ref_odf))

### Total area

In [ ]:
# add an area field to RGI_ss and reference data
RGI_ss['area'] = RGI_ss.to_crs({'proj':'cea'}).area
ref_odf['area'] = ref_odf.to_crs({'proj':'cea'}).area

In [ ]:
# print and compare area values
Area_Rep = RGI_ss['area'].sum()/1000000
print('Area Rep [km²]:', Area_Rep)
Area_RGI6 = ref_odf['area'].sum()/1000000
print('Area RGI6 [km²]:', Area_RGI6)
d = (Area_Rep - Area_RGI6)
d_perc = (d/Area_Rep*100)
print('Area difference [km²]:',d,'/','percentage:', d_perc)

### result of check (RGI from GLIMS L2 and original inventory):
#### difference in number of glaciers: 516
#### duplicate IDs: 0 in GLIMS, 438 in original inventory
#### area difference: 41 km² / 0.17 %
#### general comment: in general GLIMS ingestion works but differences need more detailed check

## ............| Extra analysis, find repeated IDs, find possible missing glaciers,... starts here |...................

#### check for duplicate glacier IDs: many glaciers have shared id in reference data set. Let's see how many of them are actually different glaciers

In [ ]:
print('Duplicate IDs in original:', len(ref_odf)-len(ref_odf['OBJECTID'].unique()))
print('Duplicate IDs in GLIMS:', len(RGI_ss)-len(RGI_ss['glac_id'].unique()))

In [ ]:
abs(len(ref_odf['OBJECTID'].unique())-len(ref_odf))

In [ ]:
## subset ref_odf to work with the subset that has not unique IDs
rep_id = gpd.geodataframe.GeoDataFrame()
for idi in ref_odf['OBJECTID'].unique():
    if len(ref_odf.loc[ref_odf['OBJECTID']==idi]) > 1:
        rep_id = pd.concat([rep_id, ref_odf.loc[ref_odf['OBJECTID']==idi]])
        print(len(ref_odf.loc[ref_odf['OBJECTID']==idi]), idi)
rep_id['OBJECTID']


### **?** I don't know why do we have length 445 but 438 repeated IDs

In [ ]:
## Repeated Ids:
tmp_ids = rep_id['OBJECTID'].unique()

## find if the glaciers with same id are actually the same glacier or not:
for tmp in tmp_ids:
    tmp_set = rep_id.loc[rep_id['OBJECTID'] == tmp]
    
## number of total glaciers with reperated IDs but different other attributes:
    print( 'Repeated Id: ', tmp, '; glaciers with this id based on "geometry": ', len(tmp_set['geometry'].unique()))
    

In [ ]:
rep_id.loc[rep_id["OBJECTID"]==4201].plot() # --> repeated ID not repeated geometry. (different geometries with same ID)
rep_id.loc[rep_id["OBJECTID"]==4721].plot() # --> repeated ID not repeated geometry. (different geometries with same ID)

In [ ]:
rep_id.loc[rep_id["OBJECTID"]==330].plot() # --> repeated ID and geometry. (glacier with ID=330 is twice in the dataset)
rep_id.loc[rep_id["OBJECTID"]==1364].plot() #--> repeated ID and geometry. (glacier with ID=1364 is twice in the dataset)

In [ ]:
rep_id.loc[rep_id["OBJECTID"]==1996].plot() # --> repeated ID not repeated geometry. (different geometries with same ID)
rep_id.loc[rep_id["OBJECTID"]==6559].plot() # --> repeated ID not repeated geometry. (different geometries with same ID)

In [ ]:
## compare all the elements inside the repeated id == 0
## many loops and not the "pythonic way", but it's how i managed to do it...
rep_id0 = rep_id.loc[rep_id["OBJECTID"] == 0]
for id0 in range(0,len(rep_id0)): ## loop all entries
    tmp = list(range(0,id0)) + list(range(id0+1,len(rep_id0))) ## all entries minus the current one
    for item in tmp: ## loop all entries except the current one
        alltrue = sum(np.array(rep_id.iloc[item]) == np.array(rep_id.iloc[id0]))
        if alltrue == 67:
            print('current equal glaciers in ids ', id0, ' and ', item)

## we don't get any message, so there are not 2 identical entries in the id=0 subset
## --> number of glaciers corresponding to id=0 is the length of the subset:
len(rep_id0)

### Check nulls in ref_odf:

In [ ]:
rep_id.isnull().sum()

In [ ]:
# Write out selection in intermediate shape files for manual GIS review
tmp_output_dir = mkdir(os.path.join(data_dir, 'l0_tmp_data', 'rgi17_inventories'))
tmp_output_dir_tar = mkdir(os.path.join(data_dir, 'l0_tmp_data'))
rep_id0.to_file(tmp_output_dir + f'/subm_{int(730):03d}_0.shp')
print('Taring...')
print(subprocess.run(['tar', '-zcvf', f'{tmp_output_dir_tar}/rgi17_inventories.tar.gz', '-C', 
                      os.path.join(data_dir, 'l0_tmp_data'), 'rgi17_inventories']))

## we have 432 glaciers with repeated id=0 (+432), 2 glaciers repeated (identical entries) (-2), 4 repeated ids that represent different polygons (2+1+1+1 = +5) --> total glaciers are "non repeated" + 432+2+5=439

### Problem: 439 vs 438 vs 432 --> look at it in more detail